In [ ]:
import time
import sqlite3
import requests
from bs4 import BeautifulSoup


# DBファイルの設定
# 保存パス（空文字 '' → 現在のフォルダに保存される）
path = ''

# 保存するSQLiteファイルの名前
db_name = 'github.db'


# スクレイピング対象URL
# GitHub のトップURL
BASE_URL = "https://github.com"

# Google が管理しているリポジトリ一覧ページ
ORG_URL = "https://github.com/google?tab=repositories"


def parse_star_count(text):
    """
    スター数の文字列を整数に変換する補助関数
    例: '1.2k' -> 1200, '12,345' -> 12345
    GitHubはスター数を "k" や "," を含めて表示するため補正が必要
    """
    text = text.strip()
    if not text:
        return 0

    # 例: "1.2k" → 1200
    if text.endswith('k'):
        try:
            num = float(text[:-1])
            return int(num * 1000)
        except ValueError:
            return 0

    # 例: "12,345" → 12345
    try:
        return int(text.replace(",", ""))
    except ValueError:
        return 0


conn = None

try:
 
    # DB接続の作成（存在しなければ自動作成）
    conn = sqlite3.connect(path + db_name)

    # SQL文を実行するためのカーソル
    cur = conn.cursor()


    # テーブルの作成
    create_sql = '''
        CREATE TABLE IF NOT EXISTS repositories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            language TEXT,
            stars INTEGER NOT NULL
        );
    '''
    cur.execute(create_sql)

    # 前回のスクレイピング結果が残っていると混ざるため、一度全削除
    cur.execute("DELETE FROM repositories;")
    conn.commit()


    # GitHub スクレイピング処理

    url = ORG_URL   # 最初のページ
    page = 1        # ページ番号カウンタ

    while url:
        print(f"\n=== PAGE {page} ===")
        print("[REQUEST]", url)

        # GitHub APIは禁止 → HTMLを requests で取得
        res = requests.get(url, headers={
            "User-Agent": "Mozilla/5.0 (student scraper for assignment)"
        })
        res.raise_for_status()

        # BeautifulSoup で HTML を解析
        soup = BeautifulSoup(res.text, "html.parser")

        # このページで取得したリポジトリ情報を入れるリスト
        repos = []

        # リポジトリ一覧の要素取得（GitHubの仕様変更に備えて2パターン）
        items = soup.select('li[itemprop="owns"]')
        if not items:
            items = soup.select("li.Box-row")

        # 1つの<li> が1つのリポジトリ
        for item in items:
            # リポジトリ名
            name_tag = (
                item.select_one('a[itemprop="name codeRepository"]')
                or item.select_one('a[data-hovercard-type="repository"]')
            )
            if not name_tag:
                continue  # 名前が取れないものはスキップ
            name = name_tag.get_text(strip=True)


            # 使用している主要な言語
            lang_tag = item.select_one('[itemprop="programmingLanguage"]')
            language = lang_tag.get_text(strip=True) if lang_tag else None


            # スター数
            star_tag = item.select_one('a.Link--muted[href*="/stargazers"]')
            if star_tag:
                stars = parse_star_count(star_tag.get_text(strip=True))
            else:
                stars = 0

            # 後でDBに保存するため、タプルにまとめてリストに追加
            repos.append((name, language, stars))

        print(f"{len(repos)} 件取得")

        # 取得したデータをSQLiteに保存

        if repos:
            insert_sql = "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)"
            cur.executemany(insert_sql, repos)
            conn.commit()
            print(f"{len(repos)} 件を保存しました")

      
        # 課題条件：各リクエストの間は必ず 1 秒空ける

        time.sleep(1)

        # 次のページを探す（旧UI / 新UI の両方に対応）
        # 旧 GitHub: a.next_page
        next_link = soup.select_one("a.next_page")
        # 新 GitHub: rel="next"
        if not next_link:
            next_link = soup.select_one("a[rel='next']")

        if next_link and next_link.get("href"):
            url = BASE_URL + next_link["href"]
            page += 1
        else:
            # 次ページが見つからなければ終了
            url = None


    # 最後に、保存したデータを SELECT で全部表示する
    print("\n=== 保存されたデータ一覧 ===")
    cur.execute("SELECT id, name, language, stars FROM repositories ORDER BY stars DESC;")

    rows = cur.fetchall()
    for row in rows:
        repo_id, name, language, stars = row
        print(f"[{repo_id}] {name} | language={language} | stars={stars}")

except Exception as e:
    print("エラーが発生:", e)

finally:
    # DB接続を閉じる
    if conn is not None:
        conn.close()



=== PAGE 1 ===
[REQUEST] https://github.com/google?tab=repositories
10 件取得
10 件を保存しました

=== 保存されたデータ一覧 ===
[43] adk-python | language=Python | stars=15612
[45] or-tools | language=C++ | stars=12715
[49] perfetto | language=C++ | stars=5021
[46] angle | language=C++ | stars=3843
[48] nomulus | language=Java | stars=1767
[50] nearby | language=C++ | stars=888
[44] osv-scalibr | language=Go | stars=538
[42] orbax | language=Python | stars=455
[41] wasefire | language=Rust | stars=129
[47] koladata | language=C++ | stars=27
